In [1]:
import numpy as np
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ratings_set_df = pd.read_csv("./data_movie_lens_100k/ratings_all_development_set.csv")
users_df = pd.read_csv("./data_movie_lens_100k/user_info.csv")
movies_df = pd.read_csv("./data_movie_lens_100k/movie_info.csv")

In [102]:
print(ratings_set_df)
print(users_df)
print(movies_df)

     user_id  age  is_male  orig_user_id
0          0   24        1             1
1          1   53        0             2
2          2   23        1             3
3          3   24        1             4
4          4   33        0             5
..       ...  ...      ...           ...
938      938   26        0           939
939      939   32        1           940
940      940   20        1           941
941      941   48        0           942
942      942   22        1           943

[943 rows x 4 columns]
      item_id                                      title  release_year  \
0           0                           Toy Story (1995)          1995   
1           1                           GoldenEye (1995)          1995   
2           2                          Four Rooms (1995)          1995   
3           3                          Get Shorty (1995)          1995   
4           4                             Copycat (1995)          1995   
...       ...                           

In [122]:
from surprise import Dataset, NormalPredictor, Reader, SVD, accuracy

svd = SVD(
    n_factors = 10,
    n_epochs=50
)

reader = Reader(rating_scale=(1,5))
ratings_df = Dataset.load_from_df(ratings_set_df[['user_id', 'item_id','rating']], reader)

trainset = ratings_df.build_full_trainset()

svd.fit(trainset)

user_vectors = svd.pu
item_vectors = svd.qi

user_features = users_df[['age', 'is_male', 'orig_user_id']].values
item_features = movies_df[['title', 'release_year']].values


user_vectors = np.resize(user_vectors, (item_features.shape[0], user_vectors.shape[1]))
item_vectors = np.resize(item_vectors, (item_features.shape[0], item_vectors.shape[1]))

user_features = np.resize(user_features, (item_features.shape[0], user_features.shape[1]))

print(user_vectors.shape)
print(item_vectors.shape)
print(user_features.shape)
print(item_features.shape)

feature_vectors = np.concatenate((user_vectors, item_vectors, user_features, item_features), axis=1)
print(feature_vectors)




(1681, 10)
(1681, 10)
(1681, 3)
(1681, 2)
[[0.3654560540258629 0.1308472728611658 0.16891049805554315 ... 1
  'Toy Story (1995)' 1995]
 [0.26832107628727037 0.15005447090513507 -0.5405755118729517 ... 2
  'GoldenEye (1995)' 1995]
 [1.2199422943036462 0.43879448845941665 0.2779406019715002 ... 3
  'Four Rooms (1995)' 1995]
 ...
 [0.2766213222286921 -0.13243244767348922 0.1605987977576589 ... 736
  'Sliding Doors (1998)' 1998]
 [0.06910087822638583 -0.0975488869642102 -0.06157610278902446 ... 737
  'You So Crazy (1994)' 1994]
 [-0.0823097210759526 -0.009388963559179633 0.030869361390236924 ... 738
  'Scream of Stone (Schrei aus Stein) (1991)' 1996]]
